<a href="https://colab.research.google.com/github/courpiaque/cordef/blob/main/trening_modelu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install label-studio-sdk --quiet
!pip install -U sacremoses --quiet
!pip install -U spacy-transformers --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.8/318.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.

In [3]:
from label_studio_sdk import Client
import pandas as pd
import spacy
from spacy.tokens import DocBin
from tqdm.notebook import tqdm
from spacy import util

In [4]:
LABEL_STUDIO_URL = 'https://app.heartex.com/'
API_KEY = 'ac32252c97f25574d668e1169b39c4d352abaaf8'

In [5]:
ls = Client(url=LABEL_STUDIO_URL, api_key=API_KEY)
ls.check_connection()

{'status': 'UP'}

In [6]:
projects = ls.get_projects()
for project in projects:
  print(f"{project.id} {project.title}")

84465 Podręczniki – Definicje bez Komunikacji społecznej
84464 Podręczniki – Komunikacja społeczna
84330 Teksty naukowe
84329 Filozofuj
84322 Wikipedia
70099 Korpus


In [7]:
project = ls.get_project(70099)

In [8]:
data = project.get_labeled_tasks()

In [9]:
len(data)

518

In [10]:
for item in data:
  text = item['data']['token']

  labels = item['annotations'][0]['result']
  for label in labels:
    label_text = label['value']['text']
    label_start = label['value']['start']
    label_end = label['value']['end']
    label_name = label['value']['labels'][0]

In [11]:
nlp = spacy.blank('pl')
docs = []

for item in data:
  text = item['data']['token']
  doc = nlp.make_doc(text)

  ents = []
  labels = item['annotations'][0]['result']
  for label in labels:
    label_text = label['value']['text']
    label_start = label['value']['start']
    label_end = label['value']['end']
    label_name = label['value']['labels'][0]
    span = doc.char_span(label_start, label_end, label=label_name, alignment_mode="contract")
    if span is None:
      print("Skipping entity")
      print(label_text, label_name)
    else:
      ents.append(span)

  doc.ents = util.filter_spans(ents)
  docs.append(doc)

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


Skipping entity
aksjologi Definiendum


In [12]:
from spacy import displacy
displacy.render(docs[-1], style="ent", jupyter=True)

In [13]:
def convert_data(DATA, output="./train.spacy"):
  nlp = spacy.blank('pl')
  db = DocBin()

  for item in DATA:
    text = item['data']['token']
    doc = nlp.make_doc(text)

    ents = []
    labels = item['annotations'][0]['result']
    for label in labels:
      label_text = label['value']['text']
      label_start = label['value']['start']
      label_end = label['value']['end']
      label_name = label['value']['labels'][0]
      span = doc.char_span(label_start, label_end, label=label_name, alignment_mode="contract")
      if span is None:
        print("Skipping entity")
        print(label_text, label_name)
      else:
        ents.append(span)
    doc.ents = util.filter_spans(ents)
    db.add(doc)
  db.to_disk(output)

In [14]:
def train_test_split(DATA, test):
    train_end = int(len(DATA) * (100-test)*0.01)
    print(train_end)
    test_start = int(len(DATA) * (100-test))+1
    return DATA[0:train_end], DATA[train_end+1:len(DATA)]

In [15]:
train_data, test_data = train_test_split(data, test=30)
test_data, dev_data = train_test_split(test_data, test=50)

362
77


In [16]:
len(dev_data)

77

In [17]:
convert_data(train_data, "./train.spacy")
convert_data(test_data, "./test.spacy")
convert_data(dev_data, "./dev.spacy")

Skipping entity
aksjologi Definiendum


In [18]:
import locale
print(locale.getpreferredencoding())
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

UTF-8


In [19]:
!python -m spacy init config --lang pl --pipeline ner config.cfg --force --optimize efficiency

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
ℹ Generated config template specific for your use case
- Language: pl
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [20]:
!python -m spacy train config.cfg --paths.train ./train.spacy  --paths.dev ./dev.spacy --output ner_model

✔ Created output directory: ner_model
ℹ Saving to output directory: ner_model
ℹ Using CPU

=========================== Initializing pipeline ===========================
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     42

In [22]:
!python -m spacy evaluate ner_model/model-best/ --output metrics.json ./test.spacy

ℹ Using CPU
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(

================================== Results ==================================

TOK     100.00
NER P   51.29 
NER R   44.40 
NER F   47.60 
SPEED   12791 


=============================== NER (per type) ===============================

                  P       R       F
Definiendum   39.06   27.47   32.26
Łącznik       82.80   88.51   85.56
Definiens     22.67   18.89   20.61

✔ Saved results to metrics.json
